In [1]:
#INITIALIZATION

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import scipy.optimize as opt
import matplotlib as mpl
import scipy.special as sf
import scipy.integrate as integ
import scipy.linalg as la
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

#if using random numbers
rng = np.random.default_rng()

In [ ]:
#DOCUMENTING A FUNCTION

#example
"""
    Returns differential equation of baseball motion from *Numerical Modeling of Baseball Pitching.*
    Intended for evaluation using scipy.integrate.solve_ivp.
    Parameters:
        t: scalar: time at which to evaluate differential equation
        y: vector: Contains [x, y, z, vx, vy, vz], the x, y, and z position (m) and x-, y-, and z-component of velocity (m/s)
        omegavec: vector: Three-component array of angular velocity in x, y, and z directions, respectively (rad/sec)
        vd: scalar: Parameter in differential equation (m/s)
        delta: scalar: Parameter in differential equation (m/s)
        f: scalar: Parameter in differential equation (m^{-1})
        B: scalar: Parameter in differential equation (rad^{-1})
    Returns:
        dydt: vector: Time derivative of each element of y, in the same order as presented.
    """

In [2]:
#ROOT FINDING
import scipy.optimize as opt

#def the function
def func_1(x):
    return 3**(-x)-x

#finding the roots using bisect
print('Bisect:')
(root_1_bisect, info_1_bisect) = opt.bisect(func_1, 0.1, 1, full_output=True, xtol=1e-14) #here, 0.1 and 1 are our a and b in bracketing
print('Function 1 Root:', root_1_bisect, 'Iterations:', info_1_bisect.iterations)

#finding the roots using brentq
print('Brentq:')
(root_1_brentq, info_1_brentq) = opt.brentq(func_1, 0.1, 1, full_output=True, xtol=1e-14)
print('Function 1 Root:', root_1_brentq, 'Iterations:', info_1_brentq.iterations)

Bisect:
Function 1 Root: 0.5478086216540996 Iterations: 47
Brentq:
Function 1 Root: 0.5478086216540975 Iterations: 7


In [ ]:
#INTERPOLATION

np.interp(x,xp,fp)
#x is the value at which to return values, ie., inputted to the interpolated fucntion
#xp: x points
#fp: f(x) points

posInterpolation = interp.make_interp_spline(x, y,k=3)
posInterpolation(2.2, nu=0)
# nu=0 0th derivitive
# this produces a function called as posInterpolation(x)

In [ ]:
#NUMERICAL DERIVATIVE

#Richardson Extrapolation
#center differencing:
def richardson_center (f, z, h, nsteps, args=()):
    """Evaluate the first derivative of a function at z, that is f'(z),
    using Richardson extrapolation and center differencing.

    Returned is the full table of approximations, Fij for j <= i. The
    values of Fij for j > i are set to zero. The final value F[-1,-1]
    should be the most accurate estimate.

    Parameters
    ----------
    f : function
        Vectorized Python function.
        This is the function for which we are estimating the derivative.
    z : number
        Value at which to evaluate the derivative.
    h : number
        Initial stepsize.
    nsteps : integer
        Number of steps to perform.
    args : tuple, optional
        extra arguments to pass to the function, f.
    """
    # Extra check to allow for args=(1) to be handled properly. This is a
    # technical detail that you do not need to worry about.
    if not isinstance(args, (tuple, list, np.ndarray)):
        args = (args,)
    # Create a zero filled table for our estimates
    F = np.zeros((nsteps, nsteps))
    # First column of F is the center differencing estimate.
    # We can fill this without a loop!
    harr = h / 2.**np.arange(nsteps)
    F[:,0] = (f(z+harr, *args) - f(z-harr, *args)) / (2.*harr)
    # Now iterate, unfortunately we do need one loop. We could
    # get rid of the inner loop but the algorithm is a little easier to
    # understand if we do not.
    for i in range(1, nsteps):
        fact = 0.25
        for j in range(1, i+1):
            F[i,j] = F[i-1,j-1] - (F[i-1,j-1] - F[i,j-1])/ (1-fact)
            fact *= 0.25
    return F

#forward differencing:
def richardson_forward (f, z, h, nsteps, args=()):
    """Evaluate the first derivative of a function at z, that is f'(z),
    using Richardson extrapolation and forward differencing.

    Returned is the full table of approximations, Fij for j <= i. The
    values of Fij for j > i are set to zero. The final value F[-1,-1]
    should be the most accurate estimate.

    Parameters
    ----------
    f : function
        Vectorized Python function.
        This is the function for which we are estimating the derivative.
    z : number
        Value at which to evaluate the derivative.
    h : number
        Initial stepsize.
    nsteps : integer
        Number of steps to perform.
    args : tuple, optional
        extra arguments to pass to the function, f.
    """

    # Create a zero filled table for our estimates
    F_forward = np.zeros((nsteps, nsteps))
    # First column of F is the forward differencing estimate.
    # We can fill this without a loop!
    harr_forward = h / 2.**np.arange(nsteps)
    F_forward[:,0] = (f(z+harr_forward, *args)-f(z, *args))/ harr_forward
    # Now iterate, unfortunately we do need one loop. We could
    # get rid of the inner loop but the algorithm is a little easier to
    # understand if we do not.
    for i in range(1, nsteps):
        fact_forward = 0.5
        for j in range(1, i+1):
            F_forward[i,j] = F_forward[i-1,j-1] - (F_forward[i-1,j-1] - F_forward[i,j-1])/ (1-fact_forward)
            fact_forward *= 0.5
    return F_forward

#implementation:
forward_der = richardson_forward(np.cos, 1.7, 0.1, 4) # where cos(x) is the function you're finding the derivative of,
#and the numbers are x, step size (h), and number of steps
center_der = richardson_center(np.cos, 1.7, 0.1, 4)

NUMERICAL DERIVATIVE FORMULAS

Forward Differencing:

$$f'(x_0) = \frac{f(x_0 + h) - f(x_0)}{h} + \mathcal{O}(h) + \cdots .$$

Richardson Extrapolation:
- First Derivative:

$$ F_1(h) = \frac{f(x_0 + h) - f(x_0)}{h} = a_1 + \alpha_1 h + \mathcal{O}(h^2) .$$

- Second Step (with q=2):

$$ F_1(h/q) = \frac{f(x_0 + h/q) - f(x_0)}{h/q} = a_1 + \alpha_1 h/q + \mathcal{O}((h/q)^2) .$$

-----------------

$$ F_1(h) = \frac{f(z+h)-f(z-h)}{2h}. $$
$$ F_1(h) = a_1 + \alpha_1 h^2. $$
$$ F_1\!\left( \frac{h}{2} \right) = \frac{f(z+h/2)-f(z-h/2)}{h}. $$
$$ a_1 = F_1(h) - \frac{F_1(h)-F_1(h/2)}{1- 1/4} \equiv F_2(h). $$
$$ a_2 = F_2(h) - \frac{F_2(h)-F_2(h/2)}{1-1/16} \equiv F_3(h). $$
$$ F_2(h/2) = F_1(h/2) - \frac{F_1(h/2)-F_1(h/4)}{1-1/4}. $$


In [ ]:
#NUMERICAL INTEGRATION

#Romberg Integration
romb_int = integ.romb(int_func, dx=x/1024) #where x is the upper bound of integral (assuming lower is 0), 
#and 1024 is (# of points to evaluate at) - 1

#Quad Integration (quad is goated)
quad_int, quad_err, quad_info = integ.quad(func, a, b, full_output=True,args=(,))
# func called as func(x,args)
# a,b are the bounds of the function

#Quad Vec
quad_vec_int,quad_err = integ.quad_vec(func, a, b, full_output=True,args=(,))
#and args must be written with a comma
# this is same as above however args can be passed with a function

In [ ]:
#ODE

#Initial Value Problems:
'''
    First need a first order system given by a function
    The function needs to take in a time value and a vector y=[x,a,b,c]
    It then needs to return the derivative of each element in a vector dydt such that dydt[0]=dx/dt=a=y[1], dydt[1]=da/dt=b=y[2], etc
    So it needs to calculate each element of dydt
    '''
solution = integ.solve_ivp(function, [1e-8, 5], init_conds, dense_output=True, vectorized=True, atol=1e-12, rtol=1e-12)
'''where [1e-8, 5] are just values of t you pick to solve across, and
    init_conds is a vector of all the initial conditions you need (for example, init_conds=[1,0] for dy_dt[0] and dy_dt[1])'''

#can also include events=function when you call the function (see PreLab05 or Lab05)
#where that function returns y[0]
np.min(solution.t_events[0]) #value of t for which the function is 0


#########################################################################################

#funcs
def function(t,y,args):
    dydt = np.zeros_like(y)
    dydt[0] = y[1]
    dydt[1] = -omega0**2 * np.sin(y[0])
    return dydt
def event1(t,y,args):
    return()

# t current timesetp
# y values for witch to calulate
# args aditional arguments
# must return an array of the same size as y
solution = integ.solve_ivp(function, [1e-8, 5], init_conds, dense_output=True, vectorized=True, atol=1e-12, rtol=1e-12,events=event1,args=(,))

#function is the function called as (t,y,args)
#function returns a vector the size of y of the system of difrential equations
#returns sol.t an array of the times 
#sol.y an array of y values for each first order difeq in the system

#event returns every spot where the given function outputs 0
#.t_events,.y_events

In [ ]:
#LINEAR ALGEBRA

A=np.array([[1,1,1],[2,2,2],[3,3,3]])
B=np.array([[1,2,3]])
C=np.array([[2, 6, 9],[5, 5, 2],[9, 4, 4]])

la.inv(C) #inverse
la.det(C) #determinent 
np.dot(A,C)==A@C #matrix multiplication

#brodcasting
print(A)
print()
print(B)
print()
print(A+B)
print()
print(A+(B.T))
print()
print(A*B)
# if the vectors are not the same size it takes the first part that would match and then does it to all subsequent parts/

def Jacobian(f, y, h, args=()):
    """
    A function that produces the Jacobian of the given system of equations
    inputs
    f the system of equations evaluated as f(y,*args)
    y the point at witch the jacobian is evaluated
    h the step sizes used for calulating derivitis 

    outputs
    JAC: a 2d array of the jacobian
    """
    y=y.reshape(-1,1)
    h_welldef=(h+y[0][0])-y[0][0]
    h_welldef=np.diag(h_welldef)
    JAC=(f(y+h_welldef,*args)-f(y-h_welldef,*args))/(2*np.diagonal(h_welldef))
    return(JAC)

#used for multidimentional root finding

[[1 1 1]
 [2 2 2]
 [3 3 3]]

[[1 2 3]]

[[2 3 4]
 [3 4 5]
 [4 5 6]]

[[2 2 2]
 [4 4 4]
 [6 6 6]]

[[1 2 3]
 [2 4 6]
 [3 6 9]]


In [ ]:
# Linear equations
la.solve(a,b)
la.lu(a)
la.lu_factor(a)
la.lu_solve(lufactors, b)
la.solve_banded((l,u),ab,b)
#this might be an important one but it all makes sense
# (l,u) number of lower and upper diagonals 
#ab banded matrix
#b 

In [ ]:
#eigenvaluesnot 
la.eig(a)
la.eigh(a)
la.eig_banded()

We can also verify that $\mathsf{B}$ diagonalizes $\mathsf{A}$. Since $\mathsf{B}$ is orthogonal we now can write this more simply as
$$ \mathsf{B}^T \mathsf{A}\mathsf{B} = \begin{pmatrix}
\lambda_1 & 0 & 0 \\
0 & \lambda_2 & 0 \\
0 & 0 & \lambda_3
\end{pmatrix}. $$

In [ ]:
# Curve fitting

def y_model(x,a1,a2,a3):
    return()
# imputs are a given x and the paramiters
# just outputs a value

(p, C) = opt.curve_fit(ymodel, x, y, sigma=sigma_y,p0=np.array([a1, a2, a3]),
                       absolute_sigma=True,args=(,))
# y model called as (x,a1-an,args)
# x,y x and y data
# sigma errors for data
# absolute sigma (use it)
#p0 inital conditons use it if the fitting fails


sigp = np.sqrt(np.diag(C))
chisq = np.sum(((y - ymodel(x, *p)) / sigma_y)**2)
dof = len(y) - len(p)
Q = sf.gammaincc(0.5*dof, 0.5*chisq)
red_chisq=chisq/dof
#this is all of the stats that can come from the fit

In [ ]:
#FFT
np.fft.fft(a,n)
# a data
# n length to transform genraly set it to nothing but if you make it longer than a it zero pads
np.fft.fftfreq(n,d)
#n=len(a)
#d=sampling rate samples per second

np.fft.fftshift(x)
#shifts arround the output of fft or fft frq
#do it
ind, = np.where(f > 0)
Hamp_sum = np.abs(H[ind]) + np.abs(H[-ind])
np.fft.irfft(a,n)

#can use r in fron
def calculate_psd(signal, sampling_rate):
    """Calculate psd for a real signal for all positive frequencies.
    Inputs:
    signal: array: Signal for which we want the psd.
    sampling_rate: float: Sampling rate of the signal.
    Outputs:
    psd: array: PSD of the signal for all f > 0
    freq: array: All the frequencies at which the psd is computed.
    """
    N = len(signal)
    fft_signal = np.fft.rfft(signal)
    freq = np.fft.rfftfreq(N, d=1/sampling_rate)
    psd = np.abs(fft_signal)**2
    psd *= 2 / (N * sampling_rate)
    # Do not include f = 0
    return psd[1:], freq[1:]

In [ ]:
# random numbers
rng.random(size=())
#uniform distribution from[0,1)
rng.normal(loc=,scale=,size=())
# random numbers on a normal distribution
#loc cetner of the dist
#scale # width of the distribution


In [ ]:
#OTHER NOTES (mostly math functions)

#fractional error calculation
frac_error = np.abs(1-(numerical_value/true_value))

#infinity
np.inf

#use np.arctan2(), not np.arctan()
np.arctan2(y,x) #where y is your numerator and x is your denominator in the tan() function
np.arctan2(np.sin(range_1), np.cos(range_1)) #example

#sums (see PreLab03 for more)
j_array = np.arange(1,13)
sum_j_function = np.sum(j_array)

#Bessel Functions (see Lab03):
import scipy.special as sf
sf.jv(nu_bessel, x_bessel)

#Elliptical Functions (see Lab04):
sf.ellipk(x)

#Indexing, Array Slicing - lots of info in PreLab05
np.where(a >= 0.5)[0] #because np.where() returns a tuple
array.copy() #do this when you're gonna change elements of an array but don't wanna fuck up the original 
#(array is just the variable name of some array)